# Setting up

Using Bayesian Optimization to fine tune hyperparameters. 4 init points and 10 iterations ran. Implemented with bayes_opt library in Python. 

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Lab/Grain-U-Net'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/MyDrive/{}/src'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/Lab/Grain-U-Net


In [ ]:
#non google colab code

FOLDERNAME = 'Lab/Grain-U-Net'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('G:/MyDrive/{}'.format(FOLDERNAME))
sys.path.append('G:/MrDrive/{}/src'.format(FOLDERNAME))
%cd G:\My Drive\Lab\Grain-U-Net

[Errno 2] No such file or directory: 'G:My DriveLabGrain-U-Net'
/content/drive/My Drive/Lab/Grain-U-Net


In [2]:
%load_ext autoreload
%autoreload 2
%aimport -tensorflow
!pip install bayesian-optimization  
!pip install -q plantcv
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=ffa93827612be5eb647a449949fe0b25682fa9e70f186a00a40e62a10a01c538
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
     |████████████████████████████████| 285 kB 5.3 MB/s 
     |████████████████████████████████| 40 kB 6.9 MB/s 
     |████████████████████████████████| 837 kB 38.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.1 MB/s 
     |████████████████████████████████| 140 kB 55.9 MB/s 
     |████████████████████████████████| 596 kB 67.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but yo

# Impoting Data

In [3]:
from pathlib import Path
import numpy as np

train_dir = Path('data/train_nouveaux_256')

input_names = list(train_dir.glob('**/image/*.png'))
label_names = list(train_dir.glob('**/label/*.png'))

test_dir = Path ('data/test_Pt/')
vali_input = list(test_dir.glob('**/raw/*.png'))
vali_label = list(test_dir.glob('**/key/*.png'))

print(f"Found {len(input_names)} samples and {len(label_names)} tracings")
print(f"Found {len(vali_input)} samples and {len(vali_label)} tracings")

Found 299 samples and 299 tracings
Found 1 samples and 1 tracings


In [4]:
import tensorflow
from src import GrainSequence
import random

def createSequences(batch_size): 
    validation_samples = len(input_names) // 10

    random.Random(1337).shuffle(input_names)
    random.Random(1337).shuffle(label_names)

    train_input = input_names[:-validation_samples]
    train_label = label_names[:-validation_samples]
    train_gen = GrainSequence(batch_size, (256,256), train_input, train_label)


    print(f"Training set size: {len(train_input)}, {len(train_gen)} batches")
    
    return train_gen

In [5]:
from skimage import io, transform

def createValidation(batch_size):

    valid_gen = GrainSequence(1, (1232, 1232), vali_input, vali_label)

    print(f"Validation set size: {len(vali_input)}, {len(valid_gen)} batches")
    img = valid_gen[0][0]
    return valid_gen


# Creating the model

In [16]:
import numpy as np 
from keras.optimizers import *
from src import get_unet_hyper, get_unet
from tensorflow import keras

def fit_with(dropout_rate, batch_size, lr=3e-4, epochs=30, verbose=1):
    epochs = int(epochs)
    batch_size = int(batch_size)
    model = get_unet_hyper(dropout_rate=dropout_rate)

    train_gen = createSequences(batch_size)

    opt = keras.optimizers.Adam(learning_rate = lr)
    model.compile(optimizer = opt, loss='binary_crossentropy', metrics = ['accuracy'])

    model_checkpoint = keras.callbacks.ModelCheckpoint('unet_grain_%.2f_%d_%.5f.hdf5' % (dropout_rate, batch_size, lr), monitor='loss', verbose=1, save_best_only=False)
    model.fit(x=train_gen, steps_per_epoch=len(train_gen), epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[model_checkpoint])

    #add in test set
    valid_gen = createValidation(batch_size)
    model2 = get_unet(input_size = (1232, 1232, 1))
    model2.load_weights('unet_grain_%.2f_%d_%.5f.hdf5' % (dropout_rate, batch_size, lr))
    print('loaded!')
    score = model2.evaluate(valid_gen, steps=len(valid_gen), verbose=0)
    print('Validation score (1-loss):' + str(1- score[0]))
    return (1 - score[0])

In [17]:
from tensorflow import keras
from bayes_opt import BayesianOptimization
import functools

pbounds = {'dropout_rate': (0.3, 0.7), 
           'batch_size': (1,10),
           'lr': (1e-5, 5e-4)
           }


In [ ]:
optimizer = BayesianOptimization(
    f=fit_with, 
    pbounds=pbounds, 
    verbose=2, 
    random_state=1
)

optimizer.maximize(init_points=4, n_iter=10)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   | batch_... | dropou... |    lr     |
-------------------------------------------------------------
Training set size: 270, 67 batches
Epoch 1/30
67/67 [==============================] - ETA: 0s - loss: 0.2492 - accuracy: 0.9384
Epoch 1: saving model to unet_grain_0.59_4_0.00001.hdf5
67/67 [==============================] - 25s 359ms/step - loss: 0.2492 - accuracy: 0.9384
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 0.2233 - accuracy: 0.9450
Epoch 2: saving model to unet_grain_0.59_4_0.00001.hdf5
67/67 [==============================] - 24s 362ms/step - loss: 0.2233 - accuracy: 0.9450
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 0.2199 - accuracy: 0.9450
Epoch 3: saving model to unet_grain_0.59_4_0.00001.hdf5
67/67 [==============================] - 25s 368ms/step - loss: 0.2199 - accuracy: 0.9450
Epoch 4/30
67/67 [==============================] - ETA: 0s - loss: 0.2172 - accuracy: 0.9450
Epoch 4: saving model to un

Validation score (1-loss):0.8739822506904602
|  1        |  0.874    |  4.753    |  0.5881   |  1.006e-0 |
Training set size: 270, 90 batches
Epoch 1/30
90/90 [==============================] - ETA: 0s - loss: 0.2342 - accuracy: 0.9449
Epoch 1: saving model to unet_grain_0.36_3_0.00006.hdf5
90/90 [==============================] - 49s 268ms/step - loss: 0.2342 - accuracy: 0.9449
Epoch 2/30
90/90 [==============================] - ETA: 0s - loss: 0.2205 - accuracy: 0.9451
Epoch 2: saving model to unet_grain_0.36_3_0.00006.hdf5
90/90 [==============================] - 26s 292ms/step - loss: 0.2205 - accuracy: 0.9451
Epoch 3/30
90/90 [==============================] - ETA: 0s - loss: 0.2135 - accuracy: 0.9451
Epoch 3: saving model to unet_grain_0.36_3_0.00006.hdf5
90/90 [==============================] - 26s 283ms/step - loss: 0.2135 - accuracy: 0.9451
Epoch 4/30
90/90 [==============================] - ETA: 0s - loss: 0.2045 - accuracy: 0.9451
Epoch 4: saving model to unet_grain_0.36_3_0

Validation score (1-loss):0.899957075715065
|  2        |  0.9      |  3.721    |  0.3587   |  5.525e-0 |
Training set size: 270, 135 batches
Epoch 1/30
135/135 [==============================] - ETA: 0s - loss: 0.2405 - accuracy: 0.9388
Epoch 1: saving model to unet_grain_0.44_2_0.00020.hdf5
135/135 [==============================] - 26s 186ms/step - loss: 0.2405 - accuracy: 0.9388
Epoch 2/30
135/135 [==============================] - ETA: 0s - loss: 0.2173 - accuracy: 0.9449
Epoch 2: saving model to unet_grain_0.44_2_0.00020.hdf5
135/135 [==============================] - 27s 199ms/step - loss: 0.2173 - accuracy: 0.9449
Epoch 3/30
135/135 [==============================] - ETA: 0s - loss: 0.2102 - accuracy: 0.9449
Epoch 3: saving model to unet_grain_0.44_2_0.00020.hdf5
135/135 [==============================] - 26s 195ms/step - loss: 0.2102 - accuracy: 0.9449
Epoch 4/30
135/135 [==============================] - ETA: 0s - loss: 0.2002 - accuracy: 0.9449
Epoch 4: saving model to unet_